#### Set-up & Main Initializations

In [ ]:
import requests
import logging
import threading
import pandas as pd
from typing import List
import time
import os
import copy
from collections import Counter
from typing import Any, Callable, Tuple
import random

import WebAPINotes.cpwalib as cpwalib
import WebAPINotes.endpoints as endpoints
import WebAPINotes.barFuncs as barFuncs
import WebAPINotes.pms as pms
import WebAPINotes.tickler as tickler
from WebAPINotes.barFuncs import Bars

import TechnicalAnalysis.techInds as Indicators

from config import globals
from config.validFields import *

logging.basicConfig(
    level=logging.INFO,  # Set log level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Include timestamp
    handlers=[
        logging.FileHandler(globals.status_file, mode='a')  # Append to the log file
    ],
    force=True
)

"""
tickle_thread = threading.Thread(target=tickler.tickle)
tickle_thread.start()
"""

""" Fixed API Data Tables to initialize as env variables """
# html tables copied from official documentation: may need update
ftdir: str = "WebAPINotes/fixedDataTables"

mdf: pd.DataFrame = pd.read_html(os.path.join(ftdir, "market_data_fields.html"))[0]
vbu: pd.DataFrame = pd.read_html(os.path.join(ftdir, "valid_bar_units.html"))[0]
vpu: pd.DataFrame = pd.read_html(os.path.join(ftdir, "valid_period_units.html"))[0]
osv: pd.DataFrame = pd.read_html(os.path.join(ftdir, "order_status_values.html"))[0]
spm: pd.DataFrame = pd.read_html(os.path.join(ftdir, "suppressible_messages.html"))[0]

#### Contract Search

In [ ]:
req_contract = cpwalib.contractSearch(
    {
        "symbol": "BTC",
        "name":   False,
    }
)

req_contract.json()

#### Saved Contracts

In [ ]:
"""

DAX INDEX

{'conid': '825711',
  'companyHeader': 'DAX 40 Index (Deutsche Aktien Xchange 40) - EUREX',
  'companyName': 'DAX 40 Index (Deutsche Aktien Xchange 40)',
  'symbol': 'DAX',
  'description': 'EUREX',
  'restricted': 'IND',
  'fop': None,
  'opt': '20240812;20240813;20240814;20240815;20240816;20240823;20240830;20240906;20240913;20240920;20240930;20241018;20241031;20241220;20250321;20250620;20250919;20251219;20260320;20260619;20260918;20261218;20270319;20270618;20271217;20281215',
  'war': '20240711;20240712;20240715;20240716;20240717;20240718;20240719;20240722;20240723;20240724;20240725;20240726;20240729;20240730;20240731;20240801;20240802;20240805;20240806;20240807;20240808;20240809;20240812;20240813;20240814;20240815;20240816;20240819;20240820;20240821;20240822;20240823;20240826;20240829;20240830;20240902;20240903;20240905;20240906;20240909;20240912;20240913;20240916;20240917;20240918;20240919;20240920;20240923;20240926;20240927;20240930;20241001;20241003;20241004;20241007;20241010;20241011;20241014;20241015;20241016;20241017;20241018;20241025;20241030;20241101;20241105;20241108;20241112;20241113;20241114;20241115;20241122;20241129;20241203;20241213;20241216;20241217;20241218;20241219;20241220;20241227;20250114;20250116;20250117;20250218;20250220;20250221;20250314;20250317;20250318;20250320;20250321;20250415;20250417;20250513;20250515;20250616;20250617;20250619;20250620;20250715;20250812;20250915;20250916;20250918;20250919;20250929;20251014;20251118;20251216;20251218;20251219;20260319;20260618',
  'sections': [{'secType': 'IND', 'exchange': 'EUREX;'},


DAX FUT

{'conid': 568953467,
  'symbol': 'DAX',
  'secType': 'FUT',
  'exchange': 'EUREX',
  'listingExchange': 'EUREX',
  'right': '?',
  'strike': 0.0,
  'currency': 'EUR',
  'cusip': None,
  'coupon': 'No Coupon',
  'desc1': "Jun20'25(5)",
  'desc2': None,
  'maturityDate': '20250620',
  'multiplier': '5',
  'tradingClass': 'FDXM',
  'validExchanges': 'EUREX'}

DAX STK

{'conid': '346727821',
  'companyHeader': 'GLOBAL X DAX GERMANY ETF - NASDAQ',
  'companyName': 'GLOBAL X DAX GERMANY ETF',
  'symbol': 'DAX',
  'description': 'NASDAQ',
  'restricted': None,
  'fop': None,
  'opt': '20240816;20240920;20241018;20250117',
  'war': None,
  'sections': [{'secType': 'STK'},
   {'secType': 'OPT',
    'months': 'AUG24;SEP24;OCT24;JAN25',
    'exchange': 'SMART;AMEX'},
   {'secType': 'BAG'}]}]


TSLA

{'conid': '76792991',
  'companyHeader': 'TESLA INC - NASDAQ',
  'companyName': 'TESLA INC',
  'symbol': 'TSLA',
  'description': 'NASDAQ',
  'restricted': None,
  'fop': None,
  'opt': '20240816;20240823;20240830;20240906;20240913;20240920;20240927;20241018;20241115;20241220;20250117;20250221;20250321;20250620;20250815;20250919;20251219;20260116;20260618;20261218',
  'war': '20240711;20240712;20240715;20240716;20240717;20240718;20240719;20240723;20240724;20240725;20240726;20240729;20240730;20240731;20240801;20240802;20240805;20240806;20240807;20240808;20240809;20240813;20240815;20240816;20240823;20240830;20240906;20240913;20240917;20240918;20240919;20240920;20241017;20241018;20241114;20241115;20241213;20241216;20241217;20241218;20241219;20241220;20250114;20250115;20250116;20250117;20250220;20250314;20250318;20250319;20250320;20250321;20250613;20250617;20250618;20250619;20250620;20250918;20250919;20251216;20251218;20251219;20260113;20260114;20260115;20260116;20260319;20260616;20260617;20260618;20260619;20260917;20261217;20261218;20270112;20270114;20270616;20270617;20271216;20271217;20280615;20280616',
  'sections': [{'secType': 'STK'}

"""

#### Contract Info

In [ ]:
# Enter the info to get specific contract
req_contract_info = cpwalib.contractInfo(
    {
        "conid": "346727821", # for STK/IND conid ONLY (default), others requrie ALL other 3 params
        "secType": "STK",  
        #"month": "JUN25",
        #"exchange": "EUREX",
        
    }
)

req_contract_info.json()

#### Contract Strike

In [ ]:
# Pick an option, and check all the strike prices available:
req_strike_info = cpwalib.contractStrikes(
    {
        "conid": "825711",
        "secType": "OPT",
        "month": "JUL24",
        "exchange": "EUREX",
        
    }
)

In [ ]:
req_strike_info.json().keys()

In [ ]:
len(req_strike_info.json()["put"])

In [ ]:
# Pick some desired strike price and feed it back into contract info search for options contract
req_options_info = cpwalib.contractInfo(
    {
        "conid": "825711",
        "secType": "OPT",
        "month": "JUL24",
        "exchange": "EUREX",
        "strike": req_strike_info.json()["put"][70],
        "right": "P"
    }
)

In [ ]:
# Now fetch the desired option:
req_options_info.json()[3]["multiplier"]

#### Alerts

In [ ]:
# Configure alert to be sent whenever order for given contract is placed
conidex: str = "346727821"
msg: str = "DAX ETF Alert Test 2"

json_content: dict = {
    "alertName": "MTA Time Alert 4",
    "alertMessage": msg,
    "alertRepeatable": 0,
    "email": "mezzacapa01@outlook.com",
    #"expireTime": "20231231-12:00:00",
    # "iTWSOrdersOnly": 0,
    "outsideRth": 1,
    "sendMessage": 1,
    # "showPopup": 1,
    "tif": "GTC",
    "conditions": [
    {
        "conidex": conidex,
        "logicBind": "n",
        "operator": ">=",
        "triggerMethod": 0,
        "type": 3, 
        "value": '20240812-14:05:00'
    }
    ]
}

resp_al = cpwalib.createAlert(globals.AccountID.FABIO.value, json_content)

resp_al.text



In [ ]:
resp_al = cpwalib.getAlerts(globals.AccountID.FABIO.value)

resp_al.json()

#### Market Data

##### Market Data Fields

In [ ]:
# Method to find field based on query substr
def find_mkdf(mdf: pd.DataFrame, query_val: str) -> pd.DataFrame:
    return mdf[mdf["Value"].apply(lambda val: query_val.lower() in val.lower())]

find_mkdf(mdf, "high")

In [ ]:
# Now locate desired field
mdf.loc[0, "Field"]

##### Market Data Snapshot

In [ ]:
""" 
The endpoint /iserver/accounts must be called 
prior to /iserver/marketdata/snapshot.
"""

acc_resp = requests.get(endpoints.base_url + endpoints.accounts, verify=False)

acc_resp.json()

In [ ]:
req_mds = cpwalib.marketDataSnapshot(
    #conids=['540729681', '673277361', '540729524'],
    conids=['825711'],
    fields=[7295, 7296, 70, 71, 87]
)

# First request does not give much out, other than repeating conids: may need further reqs
req_mds.json()

##### Historical Data 1 (/hdms endpoint)

In [ ]:
""" 
/hdms endpoint seems dead in general (BETA feature) 
"""
req_hd = cpwalib.historicalData(
    conid="825711",
    period=1,
    period_unit=PeriodUnits.DAY,
    bar=1,
    bar_unit=BarUnits.MINUTE,
    outsideRth=False,
    startTime=None,
    direction=PeriodDirecion.START_TO_NOW,
    barType=BarType.MIDPOINT
)

req_hd.text

##### Historical Data 2 (/marketdata/history endpoint)

In [ ]:

req_mdh = cpwalib.marketDataHistory(
    conid="825711",
    period=3,
    period_unit=PeriodUnits.YEAR,
    bar=1,
    bar_unit=BarUnits.DAY,
    outsideRth=True,
    startTime=None,
    exchange=None
)

req_mdh.json()


In [ ]:
time_col: str = "t"
def mdh_to_df(mdh_data: dict) -> pd.DataFrame:
    df = pd.DataFrame(mdh_data)
    # Then convert to timestamp objects (considering millisecond units)
    df.t = df.t.apply(lambda unix_time: pd.Timestamp(unix_time, unit='ms'))
    # Ensure we are sorting by increasing date 
    df = df.sort_values(by=time_col)
    return df

mdh_to_df(req_mdh.json().get("data"))

In [ ]:
# Perpetual updating algorithm
conid: str = "825711"
periods: int = 1
period_unit = PeriodUnits.DAY
bar : int = 1
bar_unit = BarUnits.MINUTE
update_time: int = 120
max_loops: int = 30

In [ ]:

# Initialize ordered data frame
curr_mdh:  pd.DataFrame = mdh_to_df(
    cpwalib.marketDataHistory(conid=conid, period=periods, period_unit=period_unit, bar=bar, bar_unit=bar_unit).json().get("data")
)

# Enter in recurring loop (needs threading)
curr_it: int = 0
while curr_it < max_loops:
    # Get freshly updated, sorted dataframe
    new_mdh: pd.DataFrame = mdh_to_df(
        cpwalib.marketDataHistory(conid=conid, period=periods, period_unit=period_unit, bar=bar, bar_unit=bar_unit).json().get("data")
    )
    # Iterate through new dataframe, adding each row if more recent than most recent (assumes that both dataframes are already sorted)
    for index, row in new_mdh.iterrows():
        if row[time_col] > curr_mdh[time_col].iloc[-1]:
            curr_mdh = pd.concat([curr_mdh, pd.DataFrame(row)], ignore_index=True)
    # Sleep until next cycle
    time.sleep(update_time)
    curr_it += 1

curr_mdh

#### Bar Import

##### BBG bar extraction

In [ ]:

raw: pd.DataFrame = pd.read_excel('dax daily.xlsx', header=6)

ref = raw[["Date", "PX_OPEN", "PX_LAST", 'PX_LOW', 'PX_HIGH']]

key_remaps: dict = {
    "Date": Bars.fields.DATE.value,
    "PX_OPEN": Bars.fields.OPEN.value,
    "PX_LAST": Bars.fields.CLOSE.value,
    "PX_LOW": Bars.fields.LOW.value,
    "PX_HIGH": Bars.fields.HIGH.value,
}

ref = ref.rename(columns=key_remaps)
#ref.to_excel("dax_daily4.xlsx")

##### IBKR extraction

In [ ]:

""" 
Full procedure to extract hourly bars for previous month
"""

mdh: pd.DataFrame = Bars.reqBars(
    conid = "825711",
    periods = 1800,
    period_unit = PeriodUnits.SECOND,
    bar = 1,
    bar_unit = BarUnits.SECOND,
    outsideRth = False,
)

mdh


In [ ]:
# Time delta minimum date difference


# if mdh.empty or Bars.fields.TIME.value not in mdh.index: continue
target_bar_size = pd.Timedelta(minutes=4)

input_bar_size : pd.Timedelta = mdh[Bars.fields.TIME.value].apply(lambda d: pd.to_datetime(d, unit=globals.UNIX_TIME_UNITS)).diff().min()
agg_fact: int = int(target_bar_size.total_seconds() / input_bar_size.total_seconds())
aggd_bars: pd.DataFrame= Bars.aggBars(
    mdh=mdh,
    agg_fact = agg_fact,
    bar_size = input_bar_size,
)

aggd_bars


In [ ]:
hourly_bars: pd.DataFrame= Bars.aggBars(
    mdh=mdh,
    agg_fact = 2,
    bar_size = pd.Timedelta(minutes=30),
)

hourly_bars.to_excel("Dax hourly bars.xlsx")

hourly_bars

##### Excel Extraction

In [ ]:
test_bars: pd.DataFrame = pd.read_excel("Dax daily bars.xlsx").drop(columns=["Unnamed: 0"])
test_bars

#### Indicators

##### Local Stationary Points

In [ ]:
# Define some bar aggregate 
from config.validFields import BarFields
from TechnicalAnalysis.techInds import BarAgg
barClose: BarAgg = lambda bar: bar[BarFields.CLOSE.value] # instance definition

In [ ]:
""" Local Stationary Points """
import TechnicalAnalysis.techInds as Indicators

# Function parameters
m = Indicators.LocStat.typ.MAX 
bar_agg = lambda i: barClose(bars.loc[i])

stat_pts_df = Indicators.LocStat().compute(bars=hourly_bars, m=m, barAgg=barClose)
stat_pts_df

In [ ]:
len(stat_pts_df)

In [ ]:
# Naive method (verification)
stat_list: List[int] = []
for i in range(1, len(hourly_bars) - 1):
    stat_list.append(i) if m.value*bar_agg(i) < m.value*bar_agg(i+1) and m.value*bar_agg(i) < m.value*bar_agg(i-1) else None
pd.Series(stat_list)

##### RSI

In [ ]:

daily_bars: pd.DataFrame= Bars.reqBars(
    conid = "825711",
    periods = 1,
    period_unit = PeriodUnits.YEAR,
    bar = 1,
    bar_unit = BarUnits.DAY,
    outsideRth = False,
)

daily_bars

In [ ]:
""" RSI """

RSI_pts: pd.DataFrame = Indicators.RSI().compute(
    bars=test_bars,
    period=14,
    barAgg=lambda bar: bar[Bars.fields.CLOSE.value]
)

RSI_pts

In [ ]:

hourly_bars: pd.DataFrame = pd.read_excel("DAX_hourly_testbars.xlsx")
hourly_bars.drop(columns="Unnamed: 0", inplace=True)
hourly_bars

##### Bollinger Bands & Crossings

In [ ]:
""" Bollinger Bands """
import TechnicalAnalysis.techInds as Indicators
BB_data : pd.DataFrame = Indicators.BollBands.compute(
    bars=hourly_bars, 
    period=20,
    mult=2.0,
    barAgg= lambda bar: bar[BarFields.CLOSE.value],
)

BB_data

In [ ]:
""" Bollinger crossings """
band = Indicators.BollBands.fields.BOLL_UPPER
dir = Indicators.BollBands.crossDir.UP

cross = lambda i: Indicators.BollBands.cross(
    bars=hourly_bars, 
    BollBands=BB_data,
    barAgg=lambda bar: bar[BarFields.CLOSE.value],
    index=i,
    band=band,
    dir=dir,
)

cross_indeces   = [i for i in hourly_bars.index if cross(i)]
cross_dates     = [hourly_bars.loc[i][BarFields.DATE.value] for i in hourly_bars.index if cross(i)]
cross_dates


##### Divergence

In [ ]:
""" Divergence """
import TechnicalAnalysis.techInds as Indicators

# Constant parameters
barClose: Indicators.BarAgg = lambda bar: bar[Bars.fields.CLOSE.value]

# Function arguments
max_div_period = 14
max_neg_period=14
#m=Indicators.LocStat.typ.MIN


In [ ]:
# Main compute method
Div_res = Indicators.Div.compute(
    test_bars, 
    barClose, 
    RSI_period=14, 
    max_div_period=max_div_period, 
    max_neg_period=max_neg_period
)

In [ ]:
Div_res

In [ ]:
# Process df before saving back results to excel
Div_res[Indicators.Div.fields.LBP.value] = Div_res[Indicators.Div.fields.LBP.value].apply(lambda i: test_bars.loc[i][Bars.fields.DATE.value])
Div_res[Indicators.Div.fields.RBP.value] = Div_res[Indicators.Div.fields.RBP.value].apply(lambda i: test_bars.loc[i][Bars.fields.DATE.value])
Div_res[Indicators.Div.fields.NEG.value] = Div_res[Indicators.Div.fields.NEG.value].apply(lambda i: test_bars.loc[i][Bars.fields.DATE.value] if i >= 0 else None)
Div_res[Indicators.Div.fields.M.value] = Div_res[Indicators.Div.fields.M.value].apply(lambda m: "max" if m==-1 else "min")

Div_res[Indicators.Div.fields.LBP.value] = Div_res[Indicators.Div.fields.LBP.value].apply(lambda t: t.strftime('%d-%m-%Y'))
Div_res[Indicators.Div.fields.RBP.value] = Div_res[Indicators.Div.fields.RBP.value].apply(lambda t: t.strftime('%d-%m-%Y'))
Div_res[Indicators.Div.fields.NEG.value] = Div_res[Indicators.Div.fields.NEG.value].apply(lambda t: t.strftime('%d-%m-%Y') if pd.notna(t) else '')

Div_res.rename(
    columns={
        Indicators.Div.fields.LBP.value: "left date",
        Indicators.Div.fields.RBP.value: "right date",
        Indicators.Div.fields.M.value: "on max / min",
    }, 
    inplace=True
)

Div_res.head(50)

In [ ]:
Div_res.to_excel("Dax Divergenze Negate Daily.xlsx")

In [ ]:
# Backtrace testing 
RBP, LBP = "right date", "left date"

i: int = 28
inter_lbps = False
div_series_list = [row for _, row in Div_res.iterrows()]

def collect_root_lbps(source_rbp: str) -> List[str]:
    """ Returns LBP for each fundamental deg. 1 divergence """
    in_divs = [pdp for pdp in div_series_list if (pdp[RBP] == source_rbp)]
    in_lbp = [pdp[LBP] for pdp in in_divs]
    if not in_divs: # Base case: if source_rbp does not stem into other incoming divergences, return this as a root
        return [source_rbp]
    return [ldp for ldp_list in (collect_root_lbps(lbp) for lbp in in_lbp) for ldp in ldp_list]

def collect_successors(source_lbp: str) -> List[str]:
    """  Returns LBP for each div. stemming and including the div. with source_lbp """
    out_divs = [pdp for pdp in div_series_list if (pdp[LBP] == source_lbp)]
    result = [out_div[LBP] for out_div in out_divs]
    for out_div in out_divs:
        result += collect_successors(out_div[RBP])
    return result


collect_root_lbps(Div_res.loc[i][RBP])

In [ ]:
collect_root_lbps("16-09-2022")

In [ ]:
collect_successors('05-09-2022')

##### Ladders 

In [ ]:
import TechnicalAnalysis.techInds as Indicators

# Must have defined test_bars first

bar_agg = lambda bar: bar[Bars.fields.CLOSE.value]
min_stat_pts = 6

req_dir = Indicators.Ladders.dir.UP

ladders = Indicators.Ladders.compute(
    test_bars, 
    bar_agg, 
    min_stat_pts=min_stat_pts
)
ladders

In [ ]:
# Process df before saving back results to excel
ladders[Indicators.Ladders.fields.LEFT_STAT.value] = ladders[Indicators.Ladders.fields.LEFT_STAT.value].apply(lambda i: test_bars.loc[i][Bars.fields.DATE.value])
ladders[Indicators.Ladders.fields.LEFT_STAT.value] = ladders[Indicators.Ladders.fields.LEFT_STAT.value].apply(lambda t: t.strftime('%d-%m-%Y'))
ladders[Indicators.Ladders.fields.RIGHT_STAT.value] = ladders[Indicators.Ladders.fields.RIGHT_STAT.value].apply(lambda i: test_bars.loc[i][Bars.fields.DATE.value])
ladders[Indicators.Ladders.fields.RIGHT_STAT.value] = ladders[Indicators.Ladders.fields.RIGHT_STAT.value].apply(lambda t: t.strftime('%d-%m-%Y'))
ladders[Indicators.Ladders.fields.DIRECTION.value] = ladders[Indicators.Ladders.fields.DIRECTION.value].apply(lambda m: "UP" if m==-1 else "DOWN")

ladders.rename(
    columns={
        Indicators.Ladders.fields.LEFT_STAT.value: "left max/min date",
        Indicators.Ladders.fields.RIGHT_STAT.value: "right max/min date",
        Indicators.Ladders.fields.DIRECTION.value: "ladder direction",
    }, 
    inplace=True
)

ladders.reset_index(drop=True, inplace=True)
ladders

In [ ]:
ladders.to_excel("Test Scale DAX Daily.xlsx")

#### Orders

##### Main Endpoint Tests

In [ ]:
# Suppress every suppressible message
req_sm = cpwalib.suppressMessages(spm.MessageId.tolist())
req_sm.text

In [ ]:
from config.globals import AccountID

req_po = cpwalib.placeOrder(
    AccountID.FABIO.value,
    [
        {
            "acctId": AccountID.FABIO.value,
            "conid": 346727821,
            "cOID": "DAX-ETF-BUY-3",
            "orderType": OrderTypes.MARKET.value,
            "side": OrderSide.BUY.value,
            "tif": Tif.IOC.value,
            "quantity": 1,
        }
    ]
)

req_po.json()

In [ ]:
# Replying to messages
req_or = cpwalib.orderReply(
    replyID=req_po.json()[0].get("id"),
    confirmed=True
)

req_or.json()

In [ ]:
# Check recent orders 

req_lo = cpwalib.liveOrders(
    filters=[
        
    ],
    force=False
)

lo_df = pd.DataFrame(req_lo.json()["orders"])

ttime: int = 240810_0000_00
# FIlter to desired columns and only recently executed orders, then sort by time
rec_trades = lo_df[lo_df['lastExecutionTime'].astype(int) > ttime][["orderId", "order_ref", "ticker", "side", "orderType", "remainingQuantity", "filledQuantity", "lastExecutionTime", "status"]].sort_values(by="lastExecutionTime", ascending=False)

# Check that filled buys equal filled sells
rec_trades[rec_trades["ticker"] == "MSFT"][rec_trades["side"] == "SELL"]["filledQuantity"].sum()

rec_trades

In [ ]:
req_lo.json()["orders"]

In [ ]:
# Check an order status
stat_req = cpwalib.orderStatus("124496446")
stat_req.json()

In [ ]:
# Cancel an order

req_co = cpwalib.cancelOrder(globals.accountID, "")
# req_co.json()

##### Sampling IOC Order Test

In [ ]:
""" Assumptions for non-crashing program
    (1) bin/run.sh root/conf.yaml script properly running to keep port 5000 open, else tickler.tickle() will throw exception and program cannot run
    (2) suppress message request suppresses all potential messages holding order execution; response object ignored
    (3) place order response object ignored
"""

# Some order constant values
conid: int      = 568953467
cOID_base: str  = "DAX-JUN25-IOCretryTest1"

# Define the fields required for the dictionary object
order_fields: List[str]=[
    "acctId", "conid", "cOID", "orderType", "side", "tif", "quantity"
]

# Define test loop variables
num_orders: int = 2                 # How many orders in total
verif_time: float = 10.0            # Time between placing IOC order and expecting it to be in live orders output
time_per_attempt: float = 10.0      # Time span over which IOCretryTillFilled attempts to fill order
max_retries: int = 4                # Maximum retries for IOC, spread over bar_size

res: List[dict] = []
res_counter = Counter()
res_orders: List[Tuple[bool, pd.DataFrame]] = []

""" NOTE: Must suppress messages beforehand, else  """
cpwalib.suppressMessages(spm.MessageId.tolist())

for order_num in range(num_orders):

    # Choose random quantity and random side
    rand_quantity: int = random.randint(1, 4)
    rand_side: str = OrderSide.BUY.value if random.randint(0, 1) == 0 else OrderSide.SELL.value

    # Try to fill the order with IOCretryTillFilled 
    res_val, order_resp = pms.LTPM.IOCretryTillFilled(
        globals.accountID,
        {
            "acctId": globals.accountID,
            "conid": conid,
            "cOID": f"{cOID_base}-{order_num}",
            "orderType": OrderTypes.MARKET.value,
            "side": rand_side,
            "tif": Tif.IOC.value,
            "quantity": rand_quantity
        },
        time_per_attempt,
        max_retries,
        verif_time
    )

    res.append({
        "side":         rand_side,
        "req_quantity": rand_quantity,
        "filled_quant": order_resp[OrderFields.FILLED_QUANTITY.value].sum() if not order_resp.empty else 0.0
    })
    res_counter[f'{res_val}'] += 1
    res_orders.append((res_val, order_resp))

res_df = pd.DataFrame(res)
print(res_counter)
res_df

In [ ]:
res_val

In [ ]:
# Analyzing resulting order dataframes
ro_cop = copy.deepcopy(res_orders)

# Redue dataframes to only rows with some fill value
non_empty_orders: List[pd.DataFrame] = [order_resp_tup[1][order_resp_tup[1][OrderFields.FILLED_QUANTITY.value] != 0] for order_resp_tup in ro_cop]

# Check the attempts and order count distributions
ord_attempts = Counter(ord_seq.iloc[0][OrderFields.ORDER_REF.value][-1] for ord_seq in non_empty_orders)
ordno_count = Counter(ord_seq.shape[0] for ord_seq in non_empty_orders)

# Display some seletc columns
non_empty_orders[41][[OrderFields.FILLED_QUANTITY.value, OrderFields.ORDER_REF.value, OrderFields.STATUS.value, OrderFields.ORDERID.value]]

#### Trades

##### Live Trading PM tests

In [ ]:
""" Open trade tests
Assumptions for non-crashing program
    (1) bin/run.sh root/conf.yaml script properly running to keep port 5000 open, else tickler.tickle() will throw exception and program cannot run
    (2) place order response object ignored
"""

from WebAPINotes.pms import Ltpm

# Some order constant values
conid: int      = 265598
cOID_base: str  = "AAPL-TradingTest7"

# Define the fields required for the dictionary object
order_fields: List[str]=[
    "acctId", "conid", "cOID", "orderType", "side", "tif", "quantity"
]

# Define test loop variables
num_trades: int = 6

# Initialize LTPM object
pm = Ltpm(
    acctId=globals.AccountID.FABIO.value,
    json_filedir="trade_data.json",
    time_per_attempt=10.0,
    max_retries=4,
    verif_time=10.0,
    spm=spm.MessageId.tolist()
)

# Attempt to initialize trade data from file if possible
pm.load_json()

trade_indexes: List[int] = []

for trade_num in range(num_trades):
    # Choose random quantity and random side
    rand_quantity: int = random.randint(1, 4)
    rand_side: str = OrderSide.BUY.value if random.randint(0, 1) == 0 else OrderSide.SELL.value

    # Define order to be passed to open trade
    req_open_order: dict = {
        "acctId": globals.AccountID.FABIO.value,
        "conid": conid,
        "cOID": f"{cOID_base}-{trade_num}",
        "orderType": OrderTypes.MARKET.value,
        "side": rand_side,
        "tif": Tif.IOC.value,
        "quantity": rand_quantity
    }

    # Simulate bar movement
    last_bar = Bars.reqBars(
        conid = conid,
        periods = 1,
        period_unit = PeriodUnits.MONTH,
        bar = 1,
        bar_unit = BarUnits.HOUR,
        outsideRth = False,
    ).iloc[trade_num + (trade_num % 2)] # Simulate bar repetition

    # Attempt to open trade with this order
    trade_index: int = pm.openTrade(
        bar=last_bar, 
        reason=f"Test trade {trade_num}", 
        req_order=req_open_order,
        block_duplicates=True,
        save_updates=True
    )

    trade_indexes.append(trade_index)

trade_indexes


In [ ]:
# Data saving tests
pm.save_to_json()
#pm.load_json()

In [ ]:
# Closing trades test
for trade_num in range(8*num_trades):
    trade_index: int = random.randint(-5, 2*num_trades)

    # Retrieve most recent bar
    last_bar = Bars.reqBars(
        conid = conid,
        periods = 1,
        period_unit = PeriodUnits.MONTH,
        bar = 1,
        bar_unit = BarUnits.HOUR,
        outsideRth = False,
    ).iloc[-1]

    pm.closeTrade(
        trade_index=trade_index,
        bar=last_bar, 
        reason=f"Test close trade at index {trade_index}", 
        save_updates=True
    )

In [ ]:
# Check how many trades still open
sum((pm.trade_data[i].close_data is None) for i in range(len(pm.trade_data)))

##### Trades endpoint tests

In [ ]:
# Note: Requires 1 call with empty return before returning data, unless set to default!
req_tr = cpwalib.trades()
tr_df = pd.DataFrame(req_tr.json())

# FIlter to desired columns and only recently executed trades, and then sort by time 
tr_df[["symbol", "trade_time", "execution_id","side", "price", "order_ref", "net_amount", "size", "sec_type", ]].sort_values(by="trade_time", ascending=False)

In [ ]:
req_tr.json()

In [ ]:
pd.Timestamp(req_tr.json()[0]['trade_time_r'], unit="ms") 

#### Portfolio

In [ ]:
req_pos = cpwalib.accountPositions(globals.accountID, 0)
pos_df = pd.DataFrame(req_pos.json())
req_pos.json()

In [ ]:
req_pos = cpwalib.accountPosition(globals.accountID, "673277361")
req_pos.json()

#### Backtesting and Live Trading Programs 

In [ ]:
""" Moved to bt.py in BT """